# Predicting Newsletter Subscription: Analyzing Player Age and Gameplay Hours

## 1. Introduction 

### 1.1 Background Information 

<p>In the gaming industry newsletters are important for keeping players up to date with new information. These letters help maintain engagement by reaching players outside of the game itself. However, not all players subscribe, and understanding what factors contribute to that can be valuable for improving engagement. <p>
    
</p>Player characteristics such as age and how many hours a person plays often influence their level of involvement in a game. Players who spend more time in a game might be more invested and therefore more likely to want updates. Age can also affect communication preferences and level of interest. By studying these factors, game companies can better understand what motivates players to stay connected and how game companies can target different groups of people. <p>



### 1.2 Big Question

What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

### 1.3 Dataset Description 

<p>The dataset used for this project is a course provided dataset for the purpose of practicing data analysis. It contains 196 rows and 7 columns. Each row represents a single player and each column represents a different variables. <p> </p> For this project, the analysis focused on the following key variables: 
-Age : A numerical value indicating how old each player is 
-Played_hours: A numerical value showing the total number of hours played in game 
-Subscribe: A binary variable indicating whether the player subscribed to the newsletter (TRUE or FALSE).<p> </p>


## 2. Methods

## 2.1 Load in Libraries and Set Seed

In order to run our analysis, we load in the Tidyverse and Tidymodels libraries. To make our analysis reproducible, we set the seed to a specific number. We use the set.seed function to do this. 

In [3]:
# load libraries
library(tidyverse)
library(tidymodels)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [4]:
# set seed
set.seed(4321)

### 2.2 Load the Data

## 3. Results